## Glove, Word2Vec 링크

Glove 다운로드 링크 : http://nlp.stanford.edu/data/glove.6B.zip 

Word2Vec 다운로드 링크 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
import csv
import re

def open_csv():
    # csv파일을 연다!
    f = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    doc_list = []

    next(csvreader)
    for f in csvreader:
        line = re.compile("[^\w]").sub(' ', f[0].lower())
        doc_list.append(line.split())

    return doc_list

doc_list = open_csv()
print(doc_list[:3])

model = Word2Vec(sentences=doc_list, size=100, window=3, min_count=3, workers=4, sg=0)

In [ ]:
#저장
model.wv.save_word2vec_format('imdb_w2v')

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input imdb_w2v --output imdb_w2v

## 이미 만들어진 모델을 가지고 와서 쓰는 법

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

token = Tokenizer()
text = "Keras tokenizer is good"
#원래는 유니크한 단어를 세야한다
word_count = len(text)
token.fit_on_texts([text])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

f = open('./drive/My Drive/glove.6B.50d.txt', encoding="utf8")

word_vectors = {}

for line in f:
    word_vector = line.split()
    word_vectors[word_vector[0]] = word_vector[1:]

print(word_vectors['apple'])

# 단어갯수 x 임베딩 벡터의 차원의 빈 행렬을 생성
matrix = np.zeros((4, 50))

# 예시용, 실제로는 word가 있을 경우에만 추가를 해줘야함! (예외처리 필요)
for word, i in token.word_index.items():
    matrix[i] = np.array(word_vectors[word])

# 레이어를 쌓아나갈 모델을 생성
model = Sequential()
# 임베딩을 위한 레이어를 생성
model.add(Embedding(4, 50, weights=[matrix]))

In [ ]:
from gensim.models import KeyedVectors

# Word2Vec을 사용할때에 (이미 딕셔너리 형태처럼 정리가 되어있음)
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

# 각 단어들에 대응하는 벡터를 저장할 배열 생성
matrix = np.zeros((word_count, 300))

# 예시용, 실제로는 word가 있을 경우에만 추가를 해줘야함! (예외처리 필요)
for word, i in token.word_index.items():
    matrix[i] = np.array(word2vec_model['word'])

# 예시일뿐 실제로 사용가능한 모델은 아님
model = Sequential()
model.add(Embedding(word_count, 100, weights=[matrix], input_length=4))
# 이후 dense layer 로 갈때 flatten layer가 필요 (차원맞추기위함)